# TP2: Modelo de Red Neuronal
___
Lucas Nahuel Raimondi

Manuel Davila

Dolores Levi

## Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import seaborn as sns

# from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l2,l1#importar l1 tambien
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch

#configuración warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tf.keras.utils.set_random_seed(90)

In [ ]:
!wget https://raw.githubusercontent.com/dlevi150/7506R-2C2023-GRUPO33/main/TP2/datasets/train.csv

--2023-12-05 22:38:05--  https://raw.githubusercontent.com/dlevi150/7506R-2C2023-GRUPO33/main/TP2/datasets/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72229641 (69M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  68.88M  80.2MB/s    in 0.9s    

2023-12-05 22:38:09 (80.2 MB/s) - ‘train.csv’ saved [72229641/72229641]



In [ ]:
!wget https://raw.githubusercontent.com/dlevi150/7506R-2C2023-GRUPO33/main/TP2/datasets/test.csv

--2023-12-05 22:38:09--  https://raw.githubusercontent.com/dlevi150/7506R-2C2023-GRUPO33/main/TP2/datasets/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11054515 (11M) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]  10.54M  30.2MB/s    in 0.3s    

2023-12-05 22:38:10 (30.2 MB/s) - ‘test.csv’ saved [11054515/11054515]



## Preprocesamiento

In [ ]:
df = pd.read_csv('train.csv')
df_filtrado = df.copy()
df_filtrado = df_filtrado[:25000]

In [ ]:
df_test = pd.read_csv('test.csv')

In [ ]:
df_filtrado['sentimiento_encodeado'] = df_filtrado['sentimiento'].map({'positivo': 1, 'negativo': 0})
df_filtrado

,ID,review_es,sentimiento,sentimiento_encodeado
0,0,Uno de los otros críticos ha mencionado que de...,positivo,1
1,1,Una pequeña pequeña producción.La técnica de f...,positivo,1
2,2,Pensé que esta era una manera maravillosa de p...,positivo,1
3,3,"Básicamente, hay una familia donde un niño peq...",negativo,0
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo,1
...,...,...,...,...
24995,24995,Esta película fue un verdadero festival de tor...,negativo,0
24996,24996,John Wayne y Albert Dekker compiten por los de...,negativo,0
24997,24997,Tarantino se comentó una vez en un melodrama d...,positivo,1
24998,24998,"AAH Sí, el programa de ejercicios fue un gran....",positivo,1


## Tokenización

In [ ]:
ds_trabajo_x = df_filtrado.drop(['sentimiento_encodeado','sentimiento','ID'], axis='columns', inplace=False)

ds_trabajo_y = df_filtrado['sentimiento_encodeado'].copy()

x_train, x_val, y_train, y_val = train_test_split(ds_trabajo_x,
                                                    ds_trabajo_y,
                                                    test_size = 0.3,
                                                    random_state = 30,
                                                    shuffle=True)

In [ ]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(x_train.review_es)

train_sequences = tokenizer.texts_to_sequences(x_train.review_es)
train_padded = pad_sequences(train_sequences, maxlen=250, padding="post", truncating="post")

val_sequences = tokenizer.texts_to_sequences(x_val.review_es)
val_padded = pad_sequences(val_sequences, maxlen=250, padding="post", truncating="post")

test_sequences = tokenizer.texts_to_sequences(df_test.review_es)
test_padded = pad_sequences(test_sequences, maxlen=250, padding="post", truncating="post")

## Entrenamiento

Usamos una capa de Embedding y LSTM que ayudaron significativamente a mejorar la prediccción.

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

embedding_dim = 300

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=250),
    LSTM(100),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
epocas = 5
batches = 10

In [ ]:
history = model.fit(
    train_padded, y_train,
    epochs=epocas,
    validation_data=(val_padded, y_val),
    batch_size=batches
)

Epoch 1/5
1750/1750 [==============================] - 1898s 1s/step - loss: 0.6956 - accuracy: 0.5079 - val_loss: 0.6910 - val_accuracy: 0.5189
Epoch 2/5
1750/1750 [==============================] - 1904s 1s/step - loss: 0.6747 - accuracy: 0.5540 - val_loss: 0.6809 - val_accuracy: 0.5320
Epoch 3/5
1750/1750 [==============================] - 1910s 1s/step - loss: 0.6136 - accuracy: 0.6155 - val_loss: 0.6106 - val_accuracy: 0.6937
Epoch 4/5
1750/1750 [==============================] - 1914s 1s/step - loss: 0.4594 - accuracy: 0.7884 - val_loss: 0.4614 - val_accuracy: 0.7975
Epoch 5/5
1750/1750 [==============================] - 1917s 1s/step - loss: 0.2835 - accuracy: 0.8906 - val_loss: 0.4566 - val_accuracy: 0.8147


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
y_val_pred = model.predict(val_padded)

threshold = 0.5
y_val_pred_classes = (y_val_pred > threshold).astype(int)

# Calcular métricas
accuracy = accuracy_score(y_val, y_val_pred_classes)
precision = precision_score(y_val, y_val_pred_classes, average='weighted')
recall = recall_score(y_val, y_val_pred_classes, average='weighted')
f1 = f1_score(y_val, y_val_pred_classes, average='weighted')

# Imprimir las métricas
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

235/235 [==============================] - 34s 147ms/step
Accuracy: 0.8147
Precision: 0.8189
Recall: 0.8147
F1 Score: 0.8142


In [ ]:
print('Historial de entrenamiento:')
print(history.history)

Historial de entrenamiento:
{'loss': [0.6956084966659546, 0.6746624708175659, 0.6136214137077332, 0.45937827229499817, 0.2835496962070465], 'accuracy': [0.5078856945037842, 0.5540000200271606, 0.6154857277870178, 0.7883999943733215, 0.8906285762786865], 'val_loss': [0.6909682154655457, 0.6809384822845459, 0.610561192035675, 0.4613909125328064, 0.4566044509410858], 'val_accuracy': [0.5189333558082581, 0.5320000052452087, 0.6937333345413208, 0.7974666953086853, 0.8146666884422302]}


## Predicción test


In [ ]:
predict = model.predict(test_padded)

269/269 [==============================] - 44s 162ms/step


In [ ]:
predict

array([[0.94032687],
       [0.04271211],
       [0.3345359 ],
       ...,
       [0.42441437],
       [0.04218835],
       [0.9944671 ]], dtype=float32)

In [ ]:
threshold = 0.5
binary_predictions = (predict > threshold).astype(int)

df_test['sentimiento'] = binary_predictions

In [ ]:
df_test

,ID,review_es,sentimiento
0,60000,La mayor virtud de esta película es su existen...,1
1,60001,"No soy un experto cinéfilo, pero pocas veces m...",0
2,60002,Si no eres un incondicional del humor estilo T...,0
3,60003,"No sé qué está pasando, si la gente se deja ll...",1
4,60004,"Pero cuando amanece,y me quedo solo,siento en ...",1
...,...,...,...
8594,68594,"Buena no, lo siguiente. Por fin un film serio ...",1
8595,68595,"Me esperaba mucho, pero que mucho, más.Guión m...",0
8596,68596,"De mal cuerpo como sensación al finalizar, de ...",0
8597,68597,Los que han añadido comentarios os lo han dich...,0


In [ ]:
df_final = df_test[["ID", "sentimiento"]]
df_final

,ID,sentimiento
0,60000,1
1,60001,0
2,60002,0
3,60003,1
4,60004,1
...,...,...
8594,68594,1
8595,68595,0
8596,68596,0
8597,68597,0


In [ ]:
df_final['sentimiento'] = df_final['sentimiento'].map({1: 'positivo', 0: 'negativo'})
df_final.head(3)

<ipython-input-19-2a490a51f0cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['sentimiento'] = df_final['sentimiento'].map({1: 'positivo', 0: 'negativo'})


,ID,sentimiento
0,60000,positivo
1,60001,negativo
2,60002,negativo


In [ ]:
df_final.loc[df_final['sentimiento'] == "positivo"]

,ID,sentimiento
0,60000,positivo
3,60003,positivo
4,60004,positivo
9,60009,positivo
10,60010,positivo
...,...,...
8590,68590,positivo
8591,68591,positivo
8592,68592,positivo
8594,68594,positivo


In [ ]:
df_final.to_csv('submission.csv', index=False)

In [ ]:
df.to_csv('/content/drive/My Drive/submission.csv', index=False)